In [1]:
# Library import

import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import pandas as pd
from pathlib import Path
import datetime as dt
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from google.cloud import bigquery
from google.oauth2 import service_account

delta=dt.timedelta(days=1)

In [2]:
# Get main path and other folders
s_mainpath=Path.cwd()
print(s_mainpath)
s_parentpath=s_mainpath.parent.parent.parent
print(s_parentpath)

sRutaPrint=s_mainpath.joinpath('Print')

c:\Alejo\Eje de Planeación\Análisis Eléctrico\ScriptsPython
c:\Alejo


In [3]:
# Información del proyecto y autenticación a BQ
project_id = "enersinc-tbsg-bq"
key_path = "C:\BigQuery\eramirez-tbsg.json"

# Cargar las credenciales del archivo JSON
credentials = service_account.Credentials.from_service_account_file(key_path)

# Crear el cliente de BigQuery
client = bigquery.Client(project=project_id, credentials=credentials)

In [4]:
# Consulta a la maestra de recursos
query = """
select DISTINCT recurso_ofei, unidad_ofei
			from tbsg.public_maestra_recurso 
			where recurso_despacho is not null and tipo_despacho='DC'
            and recurso_despacho not in ('AMOYA','CUCUANA','C_LLERAS','DARIOVS','ESCUELAMINAS','ESMERALDA','MLAGUNETA','SALTO II','SANMIGUEL','SANFRANCISCO','RPIEDRAS')
			order by recurso_ofei 
"""

# Ejecutar la consulta
df_PltU = client.query(query).to_dataframe()
# df_PltU=df_Comb[['recurso_despacho','recurso_ofei','combustible']]
# df_PltU=df_Comb.drop_duplicates()

In [77]:
# Lectura de datos de enrtrada para el cálculo
s_parentpath=Path('C:\Alejo\ExtracciónDatos')
filepath=s_parentpath.joinpath(s_parentpath,'Parametros.xlsx')

# Loas units
sheet_name='Unidad'
df_Unidad=pd.read_excel(filepath, header=0,sheet_name=sheet_name)
df_Unidad['Unidad'] = df_Unidad['Unidad'].str.replace(' ', '')
# NEPmes=NEPmes.set_index('Embalse_Sinergox')

# Load unavailability
sheet_name='MmtoUnidad'
df_Mmto=pd.read_excel(filepath, header=0,sheet_name=sheet_name)
# df_Mmto['Unidad'] = df_Unidad['Unidad'].str.replace(' ', '')
# NEPmes=NEPmes.set_index('Embalse_Sinergox')

In [78]:
df_data=df_Unidad.merge(df_PltU,left_on=['Unidad'],right_on=['unidad_ofei'], how='left')[['Unidad', 'unidad_ofei', 'CEN', 'recurso_ofei']]
df_data = df_data.dropna(subset=['unidad_ofei'])

# Dataframe with mmtos
df_Mmto=df_Mmto.merge(df_data,left_on=['unidad'],right_on=['Unidad'], how='left')[['unidad', 'fechaIni', 'fechaFin','CEN']]
df_Mmto = df_Mmto.dropna(subset=['CEN'])

In [131]:
# Rango de análisis

s_FechaIni='2025-02-10'
s_FechaFin='2025-11-30'
# s_FechaFin='2025-11-30'

#Camniar a formato fecha
d_FechaIni=dt.datetime.strptime(s_FechaIni,'%Y-%m-%d')
d_FechaFin=dt.datetime.strptime(s_FechaFin,'%Y-%m-%d')



In [137]:
# Inicializar fecha
d_Fecha=d_FechaIni
df_data1=pd.DataFrame()

while d_Fecha<=d_FechaFin:

    dfAux=df_data.copy()
    df_Mmtosdia=df_Mmto[(pd.to_datetime(df_Mmto['fechaIni'])<=d_Fecha) & (pd.to_datetime(df_Mmto['fechaFin'])>=d_Fecha)]

    for ind in df_Mmtosdia.index:
        dfAux.loc[dfAux['Unidad']==df_Mmtosdia.loc[ind,'unidad'],'CEN']=0

    dfAux['Fecha']=d_Fecha
    dfAux=dfAux.groupby(['Fecha','recurso_ofei'])[['CEN']].sum()
    dfAux=dfAux.reset_index()
    dfAux['CEN']=(dfAux['CEN']*24)/1000
    total_CEN = dfAux['CEN'].sum()
    dfAux = pd.concat([dfAux, pd.DataFrame({'Fecha': d_Fecha, 'recurso_ofei': ['Total'], 'CEN': [total_CEN]})], ignore_index=True)    

    df_data1=pd.concat([df_data1,dfAux])

    d_Fecha=d_Fecha+delta

In [ ]:
df_data2=df_data1.copy()
df_data2['Mes']=df_data2['Fecha'].dt.month
df_data2=df_data2.groupby(['recurso_ofei','Mes'])[['CEN']].sum()
df_data2=df_data2.reset_index()
df_data2


,recurso_ofei,Mes,CEN
0,ALBAN,2,194.712
1,ALBAN,3,317.688
2,ALBAN,4,307.440
3,ALBAN,5,317.688
4,ALBAN,6,307.440
...,...,...,...
215,URRA,7,249.480
216,URRA,8,249.432
217,URRA,9,241.320
218,URRA,10,249.432


In [143]:
#Print data
df_data1.to_csv(sRutaPrint.joinpath('DisponibilidadDía.csv'))
df_data1.to_csv(sRutaPrint.joinpath('DisponibilidadMes.csv'))

In [128]:
dfAux2=dfAux.copy()
total_CEN = dfAux2['CEN'].sum()
df_Aux3=pd.DataFrame({'Fecha': d_Fecha, 'recurso_ofei': ['Total'], 'CEN': [total_CEN]})
# df_Aux3 = df_Aux3.reindex(columns=dfAux2.columns, fill_value=np.nan)
dfAux2 = pd.concat([dfAux2, df_Aux3], ignore_index=True)

# Ensure the columns match before concatenation

# dfAux2 = dfAux2.reindex(columns=['Fecha', 'recurso_ofei', 'CEN'])

dfAux2

,Fecha,recurso_ofei,CEN
0,2025-02-15,ALBAN,10.248
1,2025-02-15,BETANIA,12.960
2,2025-02-15,CALIMA,3.168
3,2025-02-15,CHIVOR,12.000
4,2025-02-15,ELQUIMBO,9.600
5,2025-02-15,GUATAPE,13.440
6,2025-02-15,GUATRON,12.840
7,2025-02-15,GUAVIO,30.000
8,2025-02-15,ITUANGO,21.600
9,2025-02-15,JAGUAS,4.080


In [114]:
dfAxu2

,Fecha,recurso_ofei,CEN
0,2025-02-15,ALBAN,10.248
1,2025-02-15,BETANIA,12.960
2,2025-02-15,CALIMA,3.168
3,2025-02-15,CHIVOR,12.000
4,2025-02-15,ELQUIMBO,9.600
5,2025-02-15,GUATAPE,13.440
6,2025-02-15,GUATRON,12.840
7,2025-02-15,GUAVIO,30.000
8,2025-02-15,ITUANGO,21.600
9,2025-02-15,JAGUAS,4.080


In [115]:
df_Aux3

,Fecha,recurso_ofei,CEN
0,2025-02-16,Total,258.048
